# Testing Connection to Google Sheets

In [1]:
import numpy as np
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import jsmith_acquire
import jsmith_prepare

In [2]:
#Set up credentials
gc = gspread.service_account(filename='credentials.json')

In [3]:
#Open Google Sheet By Name
gsheet = gc.open("Testing_293rd")

In [4]:
#Try Loading First Tab, 'Original Data', as a dataframe
original_data = pd.DataFrame(gsheet.sheet1.get_all_records())

In [5]:
original_data.head()

,cause_number,notes,disposed,dispose_date,on_track,notes_lupita,file_date,cause_of_action,docket_date,docket_type,plaintiff
0,03-06-19302-MCV,Waiver of service 6/30/2003,,,,,,,,,
1,21-12-40648-MCV,WAIVER OF SERVICE 12/09/2021,,,,,12/8/2021,DIVORCE - NO CHILDREN,,,FIDEL MARTINEZ JR.
2,07-01-22238-MCV,WAIVER OF SERVICE 01/10/2022,,,,,1/9/2007,"MOTION ENFORCEMENT, TO M",5/14/2007,FINAL HEARING,
3,03-08-19404-MCV,Waive 8/11/2003,,,,,,,,,
4,15-07-31756-MCV,"UIFSA - FOREIGN SUPPORT ORDER, NOTICE OF REG. ...",FALSE,,,,7/27/2015,1,,,"DEHOYOS, MARIO ALBERT"


# Connection Successful!

Now try adding the newly created data from your functions to the other tabs.

In [6]:
#Create a list of PDF paths to iterate through
path_list = ['/Users/johnathonsmith/Downloads/CV.PEND.CASE_Dimmit_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CV.PEND.CASE_Zavala_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CV.PEND.CASE_1-24-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Zavala_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Maverick_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Dimmit_1-21-2022.pdf']

In [7]:
path_list[0]

'/Users/johnathonsmith/Downloads/CV.PEND.CASE_Dimmit_1-21-2022.pdf'

In [8]:
#Try adding just one first
df = jsmith_acquire.build_dataframe(path_list[0])
df = jsmith_prepare.prepare_dataframe(df)

Collected Data From 644 Cases.


In [9]:
df.head()

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney,Case Type,On Track,Status,File Has Image,Need File,Disposed Date,Finding,Finding Date
0,Dimmit,00-03-01927-TX,03/21/2000,TAX-BEFORE 1.1.2008,,,,,JOHN DAVID BRADY\nELIZABETH BRADY,,,,Tax,False,Pending,,,,,
1,Dimmit,00-04-01934-TX,04/20/2000,TAX SUIT,,,,,STATE OF TEXAS COUNTY OF DI,DANIEL P. O'CONNOR,ALICIA RANGEL GOMEZ,,Tax,False,Pending,,,,,
2,Dimmit,00-04-01938-TX,04/20/2000,TAX SUIT,,,,,STATE OF TEXAS COUNTY OF DI,JAMES E. CABELLO,LUCINDA LEDESMA\nPATRICIA LEDESMA\nM E GARY TR...,,Tax,False,Pending,,,,,
3,Dimmit,00-04-09284-CV,04/17/2000,"SUIT ON CONTRACTS,NOTES,",,,,,"FIRST SELECT, INC.",AUBYN JR. SHETTLE,CANDIDO R. DEANDA,,Civil,False,Pending,,,,,
4,Dimmit,00-05-01948-TX,01/19/2021,TAX DELINQUENCY,,,,,CARRIZO SPRINGS CONSOLIDATE,JAMES E. CABELLO,"GUMECINDO GUTIERREZ, ET. AL.",,Tax,False,Pending,,,,,


In [11]:
#Civil cases go to the 'Civil Cases' tab, index = 1
civil_sheet = gsheet.worksheet('Civil Cases')

#Criminal cases go to the 'Criminal Cases' tab, index = 3
crim_sheet = gsheet.worksheet('Criminal Cases')

In [12]:
civil_sheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1acypUJgOe1_KbVFL8vp5xhLBADOG8u_6AqCNkLPNCSA',
 'updatedRange': "'Civil Cases'!A1:T645",
 'updatedRows': 645,
 'updatedColumns': 20,
 'updatedCells': 12900}

In [13]:
#Create dataframes
civil_df = pd.DataFrame()
crim_df = pd.DataFrame()

In [19]:
#Use a loop to extract all the data and prepare the dataframes accordingly.
#Add them one by one to the google sheet in the proper tabs

for path in path_list:
    #Extract the PDF data
    df = jsmith_acquire.build_dataframe(path)
    
    #Prepare the df and add new columns
    df = jsmith_prepare.prepare_dataframe(df)
    
    if df['Case Type'][0] == 'Criminal':
        #Add to criminal cases tab
        crim_df = crim_df.append(df, ignore_index = True)
    elif df['Case Type'][0] == 'Civil' or df['Case Type'][0] == 'Tax':
        civil_df = civil_df.append(df, ignore_index = True)
    else:
        print('Something went wrong in the loop!')

Collected Data From 644 Cases.
Collected Data From 779 Cases.
Collected Data From 1148 Cases.
Collected Data From 59 Cases.
Collected Data From 221 Cases.
Collected Data From 77 Cases.


In [21]:
#Now add it to the proper tab in the Google Sheet
crim_sheet.update([crim_df.columns.values.tolist()] + crim_df.values.tolist())
civil_sheet.update([civil_df.columns.values.tolist()] + civil_df.values.tolist())

{'spreadsheetId': '1acypUJgOe1_KbVFL8vp5xhLBADOG8u_6AqCNkLPNCSA',
 'updatedRange': "'Civil Cases'!A1:T2572",
 'updatedRows': 2572,
 'updatedColumns': 20,
 'updatedCells': 51440}

In [23]:
crim_df.head()

,County,Cause Number,File Date,Docket Date,Defendant Name,Attorney Name,Bondsman Name,Offense,Status,Case Type,On Track,File Has Image,Need File,Disposed Date,Finding,Finding Date
0,Zavala,00-00-00000-ZCR,08/23/2019,,,,,,Pending,Criminal,False,,,,,
1,Zavala,03-12-02906-ZCR,12/30/2009,,"PUENTE, LISA MARIE",,,BURGLARY OF HABITATION,Pending,Criminal,False,,,,,
2,Zavala,04-06-02918-ZCR,01/26/2010,,"RIOS, OSVALDO",,,BURGLARY OF BUILDING,Pending,Criminal,False,,,,,
3,Zavala,04-06-02920-ZCR,01/29/2010,,"RIOS, OSVALDO","BAGLEY, MICHAEL",,BURGLARY OF BUILDING,Pending,Criminal,False,,,,,
4,Zavala,07-04-03066-ZCR,04/30/2007,,"HERRERA, JOSE NICOLAS","PADILLA, ALFRED",SAENZ BAIL BOND,INDECENCY W/CHILD SEXUAL CONTACT,Pending,Criminal,False,,,,,
